In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Outlier detection
from scipy.stats import zscore

%matplotlib inline
sns.set(style='white', context='notebook')

In [2]:
pip install scikit-learn 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

data = pd.read_excel("Data_Train.xlsx")  # assuming it's in the same directory
data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'Data_Train.xlsx'

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
data["Duration"].value_counts()

Duration
2h 50m     550
1h 30m     386
2h 45m     337
2h 55m     337
2h 35m     329
          ... 
31h 30m      1
30h 25m      1
42h 5m       1
4h 10m       1
47h 40m      1
Name: count, Length: 368, dtype: int64

In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,10683.0,9087.064121,4611.359167,1759.0,5277.0,8372.0,12373.0,79512.0


In [7]:
data.shape

(10683, 11)

## Handling Missing Values

In [8]:
data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [9]:
data['Route'] = data['Route'].ffill(axis=0)

In [10]:
data['Route'].isnull().sum()

0

In [11]:
data['Total_Stops'] = data['Total_Stops'].ffill(axis=0)
data['Total_Stops'].isnull().sum()

0

In [12]:
data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [13]:
data["Journey_day"] = pd.to_datetime(data["Date_of_Journey"],format="%d/%m/%Y").dt.day

In [14]:
data["Journey_month"] = pd.to_datetime(data["Date_of_Journey"],format="%d/%m/%Y").dt.month

In [15]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_day,Journey_month
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR â†’ DEL,22:20:00,2025-03-22 01:10:00,2h 50m,non-stop,No info,3897,24,3
1,Air India,2019-01-05 00:00:00,Kolkata,Banglore,CCU â†’ IXR â†’ BBI â†’ BLR,05:50:00,13:15:00,7h 25m,2 stops,No info,7662,5,1
2,Jet Airways,2019-09-06 00:00:00,Delhi,Cochin,DEL â†’ LKO â†’ BOM â†’ COK,09:25:00,2025-06-10 04:25:00,19h,2 stops,No info,13882,6,9
3,IndiGo,2019-12-05 00:00:00,Kolkata,Banglore,CCU â†’ NAG â†’ BLR,18:05:00,23:30:00,5h 25m,1 stop,No info,6218,5,12
4,IndiGo,2019-01-03 00:00:00,Banglore,New Delhi,BLR â†’ NAG â†’ DEL,16:50:00,21:35:00,4h 45m,1 stop,No info,13302,3,1


In [16]:
data["Dep_hour"] = pd.to_datetime(data["Dep_Time"]).dt.hour
data['Dep_min'] = pd.to_datetime(data["Dep_Time"]).dt.minute

TypeError: <class 'datetime.time'> is not convertible to datetime, at position 0

In [ ]:
data.head()

In [ ]:
# Assuming your DataFrame is named df
data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'],format = "%d/%m/%Y")
data['Day_of_Week'] = data['Date_of_Journey'].dt.day_name()
data.head()
# Observation: The Date_of_Journey column has been converted to extract the corresponding day of the week.
# This allows us to identify whether a journey occurred on a weekday or weekend,

In [ ]:
import pandas as pd

# Step 1: Clean the column — convert everything to string and strip whitespace
data['Arrival_Time'] = data['Arrival_Time'].astype(str).str.strip()

# Step 2: Handle entries missing seconds (e.g., "13:15" → "13:15:00")
def standardize_time_format(x):
    if len(x.split()) == 2:  # Format: "YYYY-MM-DD HH:MM:SS"
        return x
    elif len(x.split(":")) == 2:  # Format: "HH:MM"
        return "1900-01-01 " + x + ":00"
    elif len(x.split(":")) == 3:  # Format: "HH:MM:SS"
        return "1900-01-01 " + x
    else:
        return pd.NA  # For truly invalid entries

# Apply the formatting function
data['Arrival_Time_clean'] = data['Arrival_Time'].apply(standardize_time_format)

# Step 3: Parse the cleaned datetime
arrival_datetime = pd.to_datetime(data['Arrival_Time_clean'], errors='coerce')

# Step 4: Extract hour and minute
data['Arrival_hour'] = arrival_datetime.dt.hour
data['Arrival_min'] = arrival_datetime.dt.minute

# Optional: Drop the cleaned column if not needed
# data.drop(columns=['Arrival_Time_clean'], inplace=True)


In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:


# Assigning and converting Duration column into list
duration = list(data["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only minutes from duration

In [ ]:
data["Duration_hours"] = duration_hours
data["Duration_mins"] = duration_mins

In [ ]:
data.head()

In [ ]:
data["Airline"].value_counts()

In [ ]:
sns.catplot(y = "Price", x = "Airline", data = data.sort_values("Price", ascending = False), kind="boxen", height = 6, aspect = 3)
plt.show()

In [ ]:
Airline = data[["Airline"]]
Airline = pd.get_dummies(Airline,drop_first = True).astype(int)
Airline.head()

In [ ]:
data["Source"].value_counts()

In [ ]:

sns.catplot(y = "Price", x = "Source", data = data.sort_values("Price", ascending = False), kind="boxen", height = 4, aspect = 3)
plt.show()

In [ ]:
Source = data[["Source"]]
Source = pd.get_dummies(Source,drop_first = True).astype(int)
Source.head()

In [ ]:
data["Destination"].value_counts()

In [ ]:
Destination = data[["Destination"]]
Destination = pd.get_dummies(Destination,drop_first = True).astype(int)
Destination.head()

In [ ]:
data["Route"]

In [ ]:
data.head()

In [ ]:
data.drop(["Route", "Additional_Info"], axis = 1, inplace = True)


In [ ]:

data["Total_Stops"].value_counts()

In [ ]:
data.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)


In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
train_data = pd.concat([data,Airline,Source,Destination],axis=1)

In [ ]:
train_data.head()

In [ ]:
train_data.drop(["Dep_Time"],axis=1,inplace=True)

In [ ]:
train_data.columns

In [ ]:
train_data.shape

In [ ]:
pip install openpyxl


In [ ]:
import pandas as pd

# Load test data
test_data = pd.read_excel(r"Test_set.xlsx")

# Basic info
print("Test data Info")
print("-"*75)
print(test_data.info())

print("\nNull values:")
print("-"*75)
test_data.dropna(inplace=True)
print(test_data.isnull().sum())

# --- 🗓️ Feature Engineering: Dates ---
test_data["Journey_Date"] = pd.to_datetime(test_data["Date_of_Journey"], format="%d/%m/%Y", errors="coerce")
test_data["Journey_day"] = test_data["Journey_Date"].dt.day
test_data["Journey_month"] = test_data["Journey_Date"].dt.month
test_data.drop(["Date_of_Journey", "Journey_Date"], axis=1, inplace=True)

# --- ⏰ Departure & Arrival Time ---
test_data["Dep_Time"] = pd.to_datetime(test_data["Dep_Time"], format="%H:%M", errors="coerce")
test_data["Dep_hour"] = test_data["Dep_Time"].dt.hour
test_data["Dep_min"]  = test_data["Dep_Time"].dt.minute
test_data.drop("Dep_Time", axis=1, inplace=True)



test_data["Arrival_Time"] = pd.to_datetime(test_data["Arrival_Time"], errors="coerce")
test_data["Arrival_hour"] = test_data["Arrival_Time"].dt.hour
test_data["Arrival_min"] = test_data["Arrival_Time"].dt.minute

# --- ⏱️ Duration Fix ---
duration = test_data["Duration"].tolist()
fixed_duration = []

for dur in duration:
    parts = dur.split()
    if len(parts) == 1:
        dur = "0h " + dur if "m" in dur else dur + " 0m"
    fixed_duration.append(dur)

hours = [int(d.split("h")[0]) for d in fixed_duration]
mins = [int(d.split("m")[0].split()[-1]) for d in fixed_duration]

test_data["Duration_hours"] = hours
test_data["Duration_mins"] = mins
test_data.drop("Duration", axis=1, inplace=True)

# --- 🎯 Categorical Encoding ---
Airline      = pd.get_dummies(test_data["Airline"], drop_first=True)
Source       = pd.get_dummies(test_data["Source"], drop_first=True)
Destination  = pd.get_dummies(test_data["Destination"], drop_first=True)

# Drop less useful columns
test_data.drop(["Airline", "Source", "Destination", "Route", "Additional_Info"], axis=1, inplace=True)

# Replace Total_Stops text with numeric
stop_map = {"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}
test_data["Total_Stops"] = test_data["Total_Stops"].map(stop_map)

# Final dataframe
data_test = pd.concat([test_data, Airline, Source, Destination], axis=1)

print("\nFinal Shape of test data:", data_test.shape)


In [ ]:
test_data.columns

In [ ]:
test_data.info()

In [ ]:
train_data.shape

In [ ]:
data_test.shape

In [ ]:
train_data.columns

In [ ]:
train_data.head()

In [ ]:
data_test.columns

In [ ]:
train_data1 = train_data

In [ ]:
train_data1.shape

In [ ]:
train_data1.columns

In [ ]:
data_test.columns

In [ ]:
# drops = ["Airline","Date_of_Journey",'Source', 'Destination', 'Dep_Time','Arrival_Time']

train_data1.drop(["Day_of_Week","Duration","Airline","Date_of_Journey",'Source', 'Destination', 'Dep_Time','Arrival_Time',],axis=1,inplace=True)
train_data1.head()

In [ ]:
train_data1.drop('Airline_Trujet',axis=1,inplace=True)

In [ ]:
train_data.drop("Dep_Time",axis=1,inplace=True)

In [ ]:
train_data1.columns

In [ ]:
train_data.shape

In [ ]:
train_data1.isnull().sum()

In [ ]:
train_data1.columns

In [ ]:
train_data1.drop('Arrival_Time_clean',axis=1,inplace=True)

In [ ]:
data_test.columns

In [ ]:
data_test.drop(['Dep_hour', 'Dep_min'],axis=1,inplace=True)

In [ ]:
data_test.shape

In [ ]:
train_data1.shape

In [ ]:
data_test = data_test.rename(columns={
    'Air India': 'Airline_Air India',
    'GoAir': 'Airline_GoAir',
    'IndiGo': 'Airline_IndiGo',
    'Jet Airways': 'Airline_Jet Airways',
    'Jet Airways Business': 'Airline_Jet Airways Business',
    'Multiple carriers': 'Airline_Multiple carriers',
    'Multiple carriers Premium economy': 'Airline_Multiple carriers Premium economy',
    'SpiceJet': 'Airline_SpiceJet',
    'Vistara': 'Airline_Vistara',
    'Vistara Premium economy': 'Airline_Vistara Premium economy',
    
    'Chennai': 'Source_Chennai',
    'Delhi': 'Source_Delhi',             # This refers to 'Source_Delhi'
    'Kolkata': 'Source_Kolkata',
    'Mumbai': 'Source_Mumbai',
    
    'Cochin': 'Destination_Cochin',
    'Hyderabad': 'Destination_Hyderabad',
    'New Delhi': 'Destination_New Delhi',
},inplace=True)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [ ]:
# Drop 'Price' from X since it's the target
X_train = train_data1.drop('Price', axis=1)

# Target variable
y_train = train_data1['Price']


In [ ]:
# Make sure columns match — fill missing columns in test data if needed
for col in X_train.columns:
    if col not in data_test.columns:
        data_test[col] = 0  # Add missing columns with 0s (safe default)

# Also remove extra columns in test set not in training set
data_test = data_test[X_train.columns]


In [ ]:
# Initialize model
lr_model = LinearRegression()

# Fit on training data
lr_model.fit(X_train, y_train)


In [ ]:
y_pred = lr_model.predict(X_test)


In [ ]:
data_test.isnull().sum()

In [ ]:
data_test.head()

In [ ]:
X = train_data1.drop("Price",axis=1)
Y = train_data1["Price"]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.75)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.metrics import r2_score 

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

In [ ]:
y_predict_lr_train = lr_model.predict(x_train)
y_predict_lr_test = lr_model.predict(x_test)

In [ ]:
train_data1.isnull().sum()

In [ ]:
train_data1 = train_data1.dropna(subset=['Total_Stops'])


In [ ]:
train_data1.isnull().sum()

In [ ]:
y_predict_lr_train = lr_model.predict(x_train)
y_predict_lr_test = lr_model.predict(x_test)

In [ ]:
print(r2_score(y_train, y_predict_lr_train))
print("\n")
print(r2_score(y_test, y_predict_lr_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train)
y_predict_rf_train = rf_model.predict(x_train)
y_predict_rf_test = rf_model.predict(x_test)
print(r2_score(y_train, y_predict_rf_train))
print("\n")
print(r2_score(y_test, y_predict_rf_test))

In [ ]:
# Features and target from training data
X = train_data1.drop(['Price'], axis=1)
y = train_data1['Price']
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)


In [ ]:
# Align column order
train_data1.columns
print(train_data1.shape)

In [ ]:
print(data_test.shape)

In [ ]:
train_data1.columns

In [ ]:
data_test.columns

In [ ]:
data_test.rename(columns={
    # Airline columns
    'Air India': 'Airline_Air India',
    'GoAir': 'Airline_GoAir',
    'IndiGo': 'Airline_IndiGo',
    'Jet Airways': 'Airline_Jet Airways',
    'Jet Airways Business': 'Airline_Jet Airways Business',
    'Multiple carriers': 'Airline_Multiple carriers',
    'Multiple carriers Premium economy': 'Airline_Multiple carriers Premium economy',
    'SpiceJet': 'Airline_SpiceJet',
    'Vistara': 'Airline_Vistara',
    'Vistara Premium economy': 'Airline_Vistara Premium economy',

    # Source columns
    'Chennai': 'Source_Chennai',
    'Delhi': 'Source_Delhi',
    'Kolkata': 'Source_Kolkata',
    'Mumbai': 'Source_Mumbai',

    # Destination columns
    'Cochin': 'Destination_Kolkata',
    
    'Hyderabad': 'Destination_Hyderabad',
    'New Delhi': 'Destination_Delhi'
}, inplace=True)


In [ ]:
data_test = data_test[X.columns]

In [ ]:
data_test.columns

In [ ]:
data_test.rename(columns={
    'Destination_New Delhi': 'Destination_Delhi',
    'Destination_Cochin': 'Destination_Kolkata'
}, inplace=True)


In [ ]:
data_test.columns

In [ ]:
train_data1.columns

In [ ]:
train_data1.rename(columns={
    'Destination_New Delhi': 'Destination_Delhi',
    'Destination_Cochin': 'Destination_Kolkata'
}, inplace=True)


In [ ]:
train_data1.columns

In [ ]:
# Features and target from training data
X = train_data1.drop(['Price'], axis=1)
y = train_data1['Price']
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)

In [ ]:
data_test = data_test[X.columns]

In [ ]:
data_test.shape

In [ ]:
train_data1.shape

In [ ]:
train_data1.columns

In [ ]:
data_test.columns

In [ ]:
predicted_prices = model.predict(data_test)

# Optional: convert to a dataframe for export or analysis
import numpy as np
results = pd.DataFrame({'Predicted_Price': np.round(predicted_prices, 2)})
print(results.head())


In [ ]:
sns.pairplot(data)

In [ ]:
data['Week'] = data['Day_of_Week'].isin(['Saturday', 'Sunday'])

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(data=data, x='Week', hue='Week')

plt.title('weekend and weekday')
plt.ylabel('Number of people travelled')
plt.legend(title='Day Type', labels=['Weekday', 'Weekend'])
plt.tight_layout()
plt.show()

# Observation : The following bar graph shows that their are more people travel on 
                # weekday rather than weekend 

In [ ]:
data.groupby(['Week'])['Price'].mean()

In [ ]:
plt.Figure(figsize=(2,2))
sns.barplot(data=data, x='Week', y='Price', color='salmon', ci=None)
plt.show() 

In [ ]:
sns.pairplot(data)

# Univariate Analysis

In [ ]:
# Scatter plot 

for column in data.columns :
    plt.figure()
    sns.scatterplot(data=data, x=column, y=data.index,hue=data.Price)

In [ ]:
res = data.select_dtypes(include=np.number).columns

for column in res :
    plt.figure(figsize=(4,4))
    sns.displot(data[column],color='darkorange',kind='kde')
    # sns.displot(data[column],color='darkorange',kind='hist')

# Observation : Here we can check the price and density of flight

In [ ]:
for column in res :
    plt.figure()
    sns.boxplot(x = data[column], data = data)

In [ ]:
sns.pairplot(data=data)

In [ ]:
skew = data['Price'].skew()
print("\nSkew\n")
print(skew)

print("\nKurt\n")
kurt = data['Price'].kurt()
print(kurt)

In [ ]:
from scipy.stats import zscore
 
z_scores = zscore(data['Price'])
filtered_dataset = data[abs(z_scores) <= 3]
filtered_dataset 

for column in res :
    plt.figure()
    sns.boxplot(x = data[column], data = data)

In [ ]:
# Calculate z-scores for the 'Starting_Salary' column
print('Before outlier removal : ',data.shape)
Pricing = zscore(data['Price'])
 
# Define a threshold for identifying outliers
threshold = 3
 
# Find outliers only in the 'Starting_Salary' column
outlier_Price = data[(np.abs(Pricing) > threshold)]
 
print(f"\nNumber of outliers found in 'Price' using a threshold of {threshold}: {len(outlier_Price)}")
# print("\n",outliers_salary)
 
# Remove outliers from the dataset
dataset_cleaned = data[(np.abs(Pricing) <= threshold)]
print('\nAfter Z-Score approach (outliers in Price removed) : ',dataset_cleaned.shape)

In [ ]:
res = dataset_cleaned.select_dtypes(include=np.number).columns

for column in res :
    plt.figure(figsize=(4,4))
    sns.displot(dataset_cleaned[column],color='darkorange',kind='kde')

In [ ]:
skew = dataset_cleaned['Price'].skew()
print("\nSkew\n")
print(skew)

print("\nKurt\n")
kurt = dataset_cleaned['Price'].kurt()
print(kurt)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plotting box plot after outlier removal
sns.boxplot(data=dataset_cleaned, x='Price')  # replace with your actual column
plt.title('Box Plot after Removing Outliers')
plt.show()


In [ ]:
# cols = ['Avg. Area Number of Bedrooms']
Q1 = data['Price'].quantile(0.10)  # Q1
Q3 = data['Price'].quantile(0.90)  # Q3
IQR = Q3-Q1
dataset = data[~((data['Price']<(Q1-1.5*IQR)) | (data['Price']>(Q3+1.5*IQR)))]
print('After IQR approach : ',dataset.shape)

In [ ]:
sns.boxplot(data=dataset, x='Price')  # replace with your actual column
plt.title('Box Plot after Removing Outliers')
plt.show()

In [ ]:
res = data.select_dtypes(include=np.number).columns

for column in res :
    plt.figure(figsize=(4,4))
    sns.displot(data[column],color='darkorange',kind='kde')

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler

scaler = RobustScaler()
scaled_data = scaler.fit_transform(data[['Price']])
scaled_df = pd.DataFrame(scaled_data, columns=['Price'])
scaled_df

In [ ]:
sns.displot(data=scaled_df, x='Price', kde=True)
 
plt.title('Distribution of Scaled Original Price')
plt.xlabel('Scaled Value (0-1)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(dataset[['Price']])
scaled_df = pd.DataFrame(scaled_data, columns=['Price'])
scaled_df

sns.displot(data=scaled_df, x='Price', kde=True)
 
plt.title('Distribution of Scaled after removal of outlier in Price column')
plt.xlabel('Scaled Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show() 

In [ ]:
# from sklearn.preprocessing import RobustScaler
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# Step 1: Scale the 'Price' column using the outlier-free dataset
scaler = RobustScaler()
scaled_data = scaler.fit_transform(filtered_dataset[['Price']])

# Step 2: Convert back to DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=['Price'])

# Step 3: Plot the scaled distribution
sns.displot(data=scaled_df, x='Price', kde=True, bins=30, aspect=1.5)

# Step 4: Customize the plot
plt.title('Distribution of Scaled Price After Outlier Removal')
plt.xlabel('Scaled Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
# Step 1: Scale the 'Price' column using the outlier-free dataset
scaler = RobustScaler()
scaled_data = scaler.fit_transform(dataset[['Price']])

# Step 2: Convert back to DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=['Price'])

# Step 3: Plot the scaled distribution
sns.displot(data=scaled_df, x='Price', kde=True, bins=30, aspect=1.5)

# Step 4: Customize the plot
plt.title('Distribution of Scaled Price After Outlier Removal')
plt.xlabel('Scaled Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1 already done: outlier removal using modified IQR
# dataset is your outlier-free DataFrame

# Step 2: Apply RobustScaler
scaler = RobustScaler()
scaled_price = scaler.fit_transform(dataset[['Price']])

# Step 3: Create a scaled DataFrame for visualization or modeling
scaled_df = pd.DataFrame(scaled_price, columns=['Price'])

# Step 4: Plot the distribution
sns.displot(data=scaled_df, x='Price', kde=True, bins=30, aspect=1.5)
plt.title('Distribution of Scaled Price (After IQR Cleaning)')
plt.xlabel('Scaled Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [ ]:
# Split features and target
X_train = train_data.drop(['Price'], axis=1)
y_train = train_data['Price']

# test_data already aligned and cleaned in previous steps
X_test = test_data  # this should already match X_train columns


In [ ]:
# Initialize model
lr_model = LinearRegression()

# Fit on training data
lr_model.fit(X_train, y_train)


In [ ]:
y_pred = lr_model.predict(X_test)
